## Import Libraries 

In [24]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt 

## Clean + Preprocess Data

In [25]:
PATH = "../kaggle/data/"
# PATH = "/kaggle/input/isic-2024-challenge/"


In [26]:
df_0 = pd.read_csv(PATH + "train-metadata.csv")
cols_to_drop = ["isic_id", "patient_id", "mel_thick_mm", "iddx_full", "iddx_1", "iddx_2", "iddx_3", "iddx_4", "iddx_5", "attribution", "copyright_license", "tbp_lv_location", "tbp_lv_location_simple", "tbp_lv_dnn_lesion_confidence", "anatom_site_general", "image_type", "tbp_tile_type"]
df_0.drop(columns=cols_to_drop, inplace=True)
df_0["sex"] = df_0["sex"].replace({"male": 1, "female": 0})

# replace all NaNs with np.nan
df_0.replace("", np.nan, inplace=True)

# Drop any column with a string vallue 
for col in df_0.columns:
    if df_0[col].dtype == "object":
        df_0.drop(columns=[col], inplace=True)

/tmp/ipykernel_1867/626784201.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df_0 = pd.read_csv(PATH + "train-metadata.csv")


In [27]:
syn_mal_df = pd.read_csv("mal_samples.csv")
# Malignant Examples 
mal_df = df_0[df_0["target"] == 1]
mal_df = pd.concat([mal_df, syn_mal_df], ignore_index=True)

In [28]:
# How many entries in each table 
print("Malignant samples: ", mal_df.shape[0])
print("Benign samples: ", df_0.shape[0])

Malignant samples:  10393
Benign samples:  401059


In [29]:
df_1 = df_0.sample(n=25000)
df_1 = pd.concat([df_1, mal_df], ignore_index=True)

# clean up for training, split into train and test
train_frac = 0.8
train_size = int(train_frac * len(df_1))

train = df_1[:train_size]
x_train = df_1.drop(columns=["target"])
y_train = df_1["target"]

test = df_1[train_size:]
x_test = df_1.drop(columns=["target"])
y_test = df_1["target"]

## Define and Train Model

In [30]:
# Define the NN 
inputs = keras.Input(shape=(len(x_train.keys()),)) # One input for each feature in the dataset 

# Add two hidden layers with 64 units each and ReLU activation
x = layers.Dense(64, activation="relu")(inputs)
x = layers.Dense(32, activation="relu")(x)

# Output layer with sigmoid activation 
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="isic-v0")


In [31]:
model.compile(
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=[keras.metrics.BinaryAccuracy()],
)

history = model.fit(x_train, y_train, batch_size=128, epochs=5, validation_split=0.2)

test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Epoch 1/20


/home/ubuntu/miniconda3/envs/tf-v2/lib/python3.8/site-packages/keras/src/backend.py:5805: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


111/111 [==============================] - 1s 3ms/step - loss: 0.7006 - binary_accuracy: 0.8753 - val_loss: 0.8640 - val_binary_accuracy: 0.0000e+00
Epoch 2/20
 81/111 [====================>.........] - ETA: 0s - loss: 0.5422 - binary_accuracy: 0.8837

/home/ubuntu/miniconda3/envs/tf-v2/lib/python3.8/site-packages/keras/src/backend.py:5805: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


111/111 [==============================] - 0s 2ms/step - loss: 0.5283 - binary_accuracy: 0.8822 - val_loss: 1.1462 - val_binary_accuracy: 0.0000e+00
Epoch 3/20
111/111 [==============================] - 0s 2ms/step - loss: 0.4279 - binary_accuracy: 0.8822 - val_loss: 1.5669 - val_binary_accuracy: 0.0000e+00
Epoch 4/20
111/111 [==============================] - 0s 2ms/step - loss: 0.3730 - binary_accuracy: 0.8822 - val_loss: 2.0415 - val_binary_accuracy: 0.0000e+00
Epoch 5/20
111/111 [==============================] - 0s 2ms/step - loss: 0.3626 - binary_accuracy: 0.8822 - val_loss: 2.1178 - val_binary_accuracy: 0.0000e+00
Epoch 6/20
111/111 [==============================] - 0s 2ms/step - loss: 0.3625 - binary_accuracy: 0.8822 - val_loss: 2.1116 - val_binary_accuracy: 0.0000e+00
Epoch 7/20
111/111 [==============================] - 0s 2ms/step - loss: 0.3625 - binary_accuracy: 0.8822 - val_loss: 2.0926 - val_binary_accuracy: 0.0000e+00
Epoch 8/20
111/111 [==============================]

In [32]:
test_df = pd.read_csv(PATH + "test-metadata.csv")
isic_ids = test_df["isic_id"]

try: 
    test_df.drop(columns=cols_to_drop, inplace=True)
except KeyError: 
    pass

test_df.replace("", np.nan, inplace=True)
test_df["sex"] = test_df["sex"].replace({"male": 1, "female": 0})
test_df.dropna(inplace=True)

# Drop any column with a string vallue 
for col in test_df.columns:
    if test_df[col].dtype == "object":
        test_df.drop(columns=[col], inplace=True)

In [33]:
print(len(test_df.keys()), len(x_train.keys()))

35 35


In [34]:
preds = model.predict(test_df)

df_sub = pd.DataFrame({"isic_id": isic_ids, "target": preds[:,0]})
df_sub.to_csv("kaggle/working/submission.csv", index=False)


1/1 [==============================] - 0s 43ms/step


In [35]:
df_sub

,isic_id,target
0,ISIC_0015657,0.117048
1,ISIC_0015729,0.117048
2,ISIC_0015740,0.117048
